In [2]:
import statsmodels.api as sm
from sklearn.metrics import r2_score

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold

In [4]:
'''
train_data: Kmers6 count for DMRs Regions
train_reads: The number of reads for each DNA sequence
train_methys: The number of methylated counts in the number of reads for each DNA sequence
train_methy_level: The probability of methylation for each DNA sequence
'''
train_data = pd.read_csv('../data/Kmers6_counts_600bp.csv')
train_reads = pd.read_csv('../data/Mouse_DMRs_counts_total.csv',header = None)
train_methys = pd.read_csv('../data/Mouse_DMRs_counts_methylated.csv',header = None)
train_methy_level = pd.read_csv('../data/Mouse_DMRs_methylation_level.csv',header = None)

In [5]:
# Preprocess the dataset to be matrix 
cell_type = 5
data = train_data.as_matrix()
level = train_methy_level.as_matrix()[:,cell_type]
reads = train_reads.as_matrix()[:,cell_type]
methy = train_methys.as_matrix()[:,cell_type]
# Add a constant to data matrix
data = np.concatenate([data,np.ones([data.shape[0],1])],axis = 1)
print('Shape of data:',data.shape)
print(level.shape,reads.shape,methy.shape)
Binomial_Target = np.concatenate([reads.reshape(-1,1),methy.reshape(-1,1)],axis=1)
print(Binomial_Target.shape)

Shape of data: (58959, 2081)
(58959,) (58959,) (58959,)
(58959, 2)


In [6]:
kf = KFold(n_splits=5)
train_score = []
test_score = []
i = 0
for train_index, test_index in kf.split(data):
    
    print('Epoch: ', i+1)
    
    X_train, X_test = data[train_index],data[test_index]
    M_train, M_test = methy[train_index],methy[test_index]
    R_train, R_test = reads[train_index],reads[test_index]
    L_train, L_test = level[train_index],level[test_index]
    print('Finish preparing for data...')
    Binomial_Target = np.concatenate([M_train.reshape(-1,1),R_train.reshape(-1,1) - M_train.reshape(-1,1)], axis = 1)
    Binomial_Model = sm.GLM(Binomial_Target,X_train, family=sm.families.Binomial())
    results = Binomial_Model.fit()
    
    # Test the prediction results
    Test = Binomial_Model.predict(results.params,X_test)
    Train = Binomial_Model.predict(results.params,X_train)
    
    # 
    Train_score = r2_score(L_train,Train)
    Test_score = r2_score(L_test,Test)
    
    i+=1
    print('Train R2 score: ', Train_score)
    print('Test R2 score: ', Test_score)
    
    num_coef = 0
    for p in results.params:
        if p != 0:
            num_coef += 1
    print("Nonzero Coefficients: ", num_coef)
    
    train_score.append(Train_score)
    test_score.append(Test_score)

Epoch:  1
Finish preparing for data...
Train R2 score:  0.2884570712747414
Test R2 score:  0.24550630174756782
Nonzero Coefficients:  2081
Epoch:  2
Finish preparing for data...
Train R2 score:  0.2969549817599214
Test R2 score:  0.218720332946847
Nonzero Coefficients:  2081
Epoch:  3
Finish preparing for data...
Train R2 score:  0.29573304597459626
Test R2 score:  0.22016581176189043
Nonzero Coefficients:  2081
Epoch:  4
Finish preparing for data...
Train R2 score:  0.2952352364273755
Test R2 score:  0.2137941816521527
Nonzero Coefficients:  2081
Epoch:  5
Finish preparing for data...
Train R2 score:  0.2965431353919419
Test R2 score:  0.21393769105401406
Nonzero Coefficients:  2081


In [9]:
print("Training Accuarcy",np.mean(train_score))
print("Testing Accuarcy",np.mean(test_score))

Training Accuarcy 0.29458469416571526
Testing Accuarcy 0.22242486383249443


In [ ]:
Binomial_Target = np.concatenate([methy.reshape(-1,1),reads.reshape(-1,1) - methy.reshape(-1,1)], axis = 1)
Binomial_Model = sm.GLM(Binomial_Target,data, family=sm.families.Binomial())
results = Binomial_Model.fit()
prediction = Binomial_Model.predict(results.params,data)

In [17]:
score = r2_score(level,prediction)
print("Score:",score)
print(results.summary())

Score: 0.28959087760650737
                 Generalized Linear Model Regression Results                  
Dep. Variable:           ['y1', 'y2']   No. Observations:                58959
Model:                            GLM   Df Residuals:                    56878
Model Family:                Binomial   Df Model:                         2080
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -6.7696e+05
Date:                Sun, 13 Jan 2019   Deviance:                   1.1277e+06
Time:                        14:20:13   Pearson chi2:                 1.49e+04
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0541      0.003    -16.648      0.000      -0.060      -0.048
x2            -0.2186    

In [15]:
# Save params out
np.save("GLM_Binomial.npy",np.array(results.params))

In [16]:
print(prediction)

[0.60060303 0.55854253 0.71541327 ... 0.59772522 0.68553947 0.18382603]


In [18]:
print(level)

[0.80769 0.98    0.12    ... 0.47059 0.91473 0.62621]
